Naudojamos **requests** biblioteka puslapiui užkrauti ir  **BeautifulSoup** HTML analizei atlikti

Sukuriamas grožinės literatūros kategorijų sąrašas

In [4]:
import requests
from bs4 import BeautifulSoup
import time

url = "https://www.knygos.lt/lt/knygos/zanras/grozine-literatura/"
response = requests.get(url)

if response.status_code == 200:
    soup = BeautifulSoup(response.text, "html.parser")
    genres = soup.find_all("a", class_="title-link")
    genres_list = []
    for genre in genres:
        genre.text.strip()
        genres_list.append(genre.get("href").split("/")[-2])  # pridedamas prie žanrų sąrašo
        time.sleep(1)  # Pridedame laukimo laiką
else:
    print("Nepavyko užkrauti puslapio")

In [5]:
genres_list

['isskirtine-knygos-lt-kolekcija',
 'meiles-romanai',
 'detektyvai-trileriai',
 'klasika',
 'uzsienio-proza',
 'lietuviu-proza',
 'fantastika-ir-fantasy',
 'tikros-istorijos',
 'istoriniai-romanai',
 'poezija-drama-ese',
 'folkloras-ir-mitologija',
 'humoras-ir-komiksai',
 'lgbt-romanai',
 'naujausi-balto-trileriai',
 'populiariausios-balto-knygos-patogiu-kiseniniu-formatu']

Pašalinamas pirmas elementas, nes ši kategoria sudaryta iš knygų rinkinių

In [6]:
genres_list.pop(0)

'isskirtine-knygos-lt-kolekcija'

Surasti, kiek kiekviena kategorija turi puslapių

In [7]:
genres_info = []
for genre in genres_list:
    response = requests.get(f"https://www.knygos.lt/lt/knygos/zanras/{genre}/")
    soup = BeautifulSoup(response.text, "html.parser")

    # Ieškome puslapiavimo elemento
    pagination = soup.find(class_="numbers d-flex")

    if pagination:
        # Jei puslapiavimas rastas, imame priešpaskutinį elementą, kuris rodo puslapių skaičių
        psl_sk = int(pagination.find_all("li")[-2].text)
        print(f"Žanras: {genre}, Puslapių skaičius: {psl_sk}")
    else:
        print(f"Žanras: {genre}, Puslapiavimo elementas nerastas.")
    genres_info.append({"name": genre, "pages": psl_sk})
    time.sleep(1)

Žanras: meiles-romanai, Puslapių skaičius: 90
Žanras: detektyvai-trileriai, Puslapių skaičius: 63
Žanras: klasika, Puslapių skaičius: 31
Žanras: uzsienio-proza, Puslapių skaičius: 287
Žanras: lietuviu-proza, Puslapių skaičius: 63
Žanras: fantastika-ir-fantasy, Puslapių skaičius: 40
Žanras: tikros-istorijos, Puslapių skaičius: 8
Žanras: istoriniai-romanai, Puslapių skaičius: 21
Žanras: poezija-drama-ese, Puslapių skaičius: 62
Žanras: folkloras-ir-mitologija, Puslapių skaičius: 4
Žanras: humoras-ir-komiksai, Puslapių skaičius: 5
Žanras: lgbt-romanai, Puslapių skaičius: 2
Žanras: naujausi-balto-trileriai, Puslapių skaičius: 1
Žanras: populiariausios-balto-knygos-patogiu-kiseniniu-formatu, Puslapių skaičius: 1


Nuskaitomos grožinės literatūros knygų nuorodos į tekstinį failą

In [ ]:
# 1 variantas with apačioje
import requests
from bs4 import BeautifulSoup
import time

# Pereiti per kiekvieną žanrą ir generuojame nuorodas
for genre in genres_info:
    genre_name = genre["name"]
    page_count = genre["pages"]
    print(genre_name, page_count)
    for page in range(1, page_count + 1):
        url = f"https://www.knygos.lt/lt/knygos/zanras/{genre_name}/?psl={page}/"
        # url = f"https://www.knygos.lt/lt/knygos/zanras/meiles-romanai/"
        response = requests.get(url)
        soup = BeautifulSoup(response.text, 'html.parser')
        books = soup.find_all("a", class_="product-link")

        time.sleep(1)
        for book in books:
            link = book.get("href").strip()
            if link:
                with open("book_links.txt", "a+") as failas:
                    failas.write(f"{link}\n")
        time.sleep(1)

print('Nuskaitymas baigtas. Knygų nuorodos išsaugotos į failą')

In [ ]:
# 2 variantas with viršuje
import requests
from bs4 import BeautifulSoup
import time

output_file = "book_links.txt"

with open(output_file, "a", encoding="utf-8") as file:
    for genre in genres_info:
        genre_name = genre["name"]
        page_count = genre["pages"]
        # print(genre_name, page_count)
        for page in range(1, page_count + 1):
            url = f"https://www.knygos.lt/lt/knygos/zanras/{genre_name}/?psl={page}/"
            response = requests.get(url)
            soup = BeautifulSoup(response.text, "html.parser")
            books = soup.find_all("a", class_="product-link")

            time.sleep(1)
            for book in books:
                link = book.get("href").strip()
                if link:
                    file.write(f"{link}\n")
            time.sleep(1)

print("Nuskaitymas baigtas. Knygų nuorodos išsaugotos į failą")